# 第５章　点群からの物体認識

## 5.1 特定物体認識と一般物体認識

- 狭義の物体認識
    - 入力データに対して、ラベルを出力する識別タスクを示すことが多い
    - 一般物体認識
        - ラベルはあらかじめ人間が決める
        - カテゴリレベルの認識タスク
            - ラベルと物体が一対一対応している
    - 特定物体認識
        - データベースに対して入力データと一致するモデルを特定する
        - インスタンスレベルの認識
            - ラベルと物体が一対多の対応なっている
  
手順
1. ラベルが付与された3次元データを用意する
2. 3次元データから特徴量を抽出する
3. 特徴量からラベルを推定する識別器を用意（学習）する
4. 識別対象物体の3次元データから特徴量を抽出する
5. 識別器を用いてラベルを推定する

In [4]:
#手順1：ラベルが付与された3次元データを用意する
import os

dirname = "rgbd-dataset"
classes = ["apple", "banana", "camera"]
url = "https://rgbd-dataset.cs.washington.edu/dataset/rgbd-dataset_pcd_ascii/"
#有名な3次元点群データセットRGB-D Object Datasetの一部をダウンロードする
for i in range(len(classes)):
    if not os.path.exists(dirname + "/" + classes[i]):
        os.system("wget " + url + classes[i] + "_1.tar")
        os.system("tar xvf "+ classes[i] + "_1.tar")

In [5]:
"""
手順2:3次元データから特徴量を抽出する
手順3:特徴量からラベルを推定する識別器を用意（学習）する -> 今回は簡略化して、k=1のk最近傍法を用いる。
すなわち認識対象物体の特徴量とデータベース内の全物体の特徴量との類似度を計算し、最も類似度の高いデータベース物体のラベルを出力するという仕組み
手順4:識別対象物体の3次元データから特徴量を抽出する
"""
import open3d as o3d
import numpy as np

"""
今回は点群データ全体で一つの物体として認識するために、点群データ全体から大域特徴量を抽出する必要がある。
そのため、 FPTHで33次元の特徴量を計算して、その総和をとって、13行目で特徴量のノルムを1に正規化して返す。
"""
def extract_fpth(filename):
    print(" ", filename)
    #点群の読み込み
    pcd = o3d.io.read_point_cloud(filename)
    #ボクセルのダウンサンプリング
    pcd = pcd.voxel_down_sample(0.01)
    #法線ベクトルの推定
    pcd.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=10))
    #FPTH特徴量を抽出
    fpth = o3d.pipelines.registration.compute_fpfh_feature(pcd, 
                                                          search_param = o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=100))
    #33次元のFPTHの特徴量の総和を計算
    sum_fpth = np.sum(np.array(fpth.data), 1)
    #最後に1に正規化して返す
    return(sum_fpth / np.linalg.norm(sum_fpth))

#### データについて
apple, banana, cameraの3個の物体の点群データに対して、それぞれ方位角と3パターンの高度を変化させて撮影したデータ600個以上入っている  
appe_1_x_y.pcdだったらxが高度角のレベルでyが方位角のレベルを示している

In [6]:
#100個を学習データ、別の100個をテキストデータとして読み込んで特徴量を抽出している
nsamp = 100
feat_train = np.zeros((len(classes), nsamp, 33))
feat_test = np.zeros((len(classes), nsamp, 33))

for i in range(len(classes)):
    print("Extracting train features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + "_1/" + classes[i] + "_1_1_" + str(n+1) + ".pcd"
        #学習データの特徴量を格納
        feat_train[i, n] = extract_fpth(filename)
    print("Exrtacting test features in " + classes[i] + "...")
    for n in range(nsamp):
        filename = dirname + "/" + classes[i] + "/" + classes[i] + "_1/" + classes[i] + "_1_4_" + str(n+1) + ".pcd"
        #学習データの特徴量を格納        
        feat_test[i, n] = extract_fpth(filename)

Extracting train features in apple...
  rgbd-dataset/apple/apple_1/apple_1_1_1.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_2.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_3.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_4.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_5.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_6.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_7.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_8.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_9.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_10.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_11.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_12.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_13.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_14.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_15.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_16.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_17.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_18.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_19.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_20.pcd
  rgbd-dataset/apple/apple_1/apple_1_1_21.pcd
  rgb

In [7]:
"""
手順5:識別器を用いてラベルを推定する -> k最近傍法を用いる
2つの点群データの類似度としてベクトルの内積を用いる
"""
for i in range(len(classes)):
    max_sim = np.zeros((3, nsamp))
    for j in range(len(classes)):
        sim = np.dot(feat_test[i], feat_train[j].transpose())
        #j番目の物体の全データの中で最も近いデータとの類似度が格納される→これが最も高いクラスとして推定されたラベルとなる
        max_sim[j] = np.max(sim, 1)
    #本来のラベルと一致している数を計算
    correct_num = (np.argmax(max_sim, 0) == i).sum()
    print("Accuracy of ", classes[i], ":", correct_num*100/nsamp, "%")

Accuracy of  apple : 98.0 %
Accuracy of  banana : 89.0 %
Accuracy of  camera : 83.0 %


### この結果からわかること
比較的容易なタスクである特定物体認識は同一の物体でも環境や撮影状況が異なるだけで精度が変化する  
  
精度が変化する要因
- 照明環境の違いによる色の変化
- センサから物体までの距離の違いによる解像度の変化
- センサノイズ
- 前景に存在する障害物による隠れ（オクルージョン）
- 物体の姿勢変化
    - 今回は特にこれが影響した
    - FPTHを使用したため、回転不変の特徴量なはず
    - だが今回は、単一視点から撮影したデータであるため、裏側が見えず2.5次元データになっていた
        - そのため点群データの存在する物体の表面が変化して、データが変化してしまった

#### 一般物体認識
- ラベルと物体が一対多の対応になっている
    - カテゴリ内の物体に共通する、より抽象的な特徴を捉えることがカギとなる
    - ただ最近の深層学習はタスクに合わせた特徴抽出を学習するため、あまり特定物体認識と一般物体認識の区別があまりない
  
今回はそのうち2例を紹介する  
- ある未知の3次元物体データが与えられたとき、形状が類似した3次元物体データをデータベースから検索し、類似度の高い順に検索結果を出力するタスク
    - 入力のカテゴリと一致した場合は正解
    - 精度と再現率を算出
    - この方法をBag-Of-Features(B0F)という
  
BoFの手順
1. データベース内の物体データから点をサンプリングする
2. 各点における局所特徴量を抽出する
    - SIFT(Scale-Invariant Feature Transform)特徴量など
3. 全ての局所特徴量に対して、k-meansクラスタリングを行う
4. 各クラスタ中心(visual word)の集合をvisual codebookとする
5. 入力データから点をサンプリング
6. 各点における局所特徴量を抽出し、最近傍のvisual wordに割り当てる
7. 各visual wordに割り当てられた点の個数を数え上げ、ヒストグラムを作る
  
大渕らの研究 
- 主成分分析を用いて三次元モデルの主軸を求め、姿勢を正規化した上で複数視点からの画像をレンダリングし、それらの画像からSIFT特徴量を抽出する
    - そして、異なる二つの物体データのBoF特徴量の類似度をKLダイバージェンスによって求める
- ヒストグラム類似度としては、カイ２条距離などで類似度を求める
  
Laiらの研究
- 木構造で徐々にカテゴリからインスタンス、さらに視点、姿勢と絞っていくことで特定する

## 5.2 特定物体の姿勢推定
- 異なる視点で撮影された点群を入力として、これらを貼り合わせる
    - 特徴点マッチング
        - 入力の二つの点群を貼り合わせるために変換行列を求めること
        - 変換行列は4×4の同時変換行列T = [R, t ; 0, 1]で表されることが一般的
        - 特徴点マッチングは初期位置が近いことを前提としていないため、より一般的なシーンで利用できる利点がある
            - ICPアルゴリズムは初期位置が近いことを前提
        - より精度が求められる状況下では、特徴点マッチングで得られた位置姿勢を初期値として、ICPアルゴリズムを適用する
- ソースとターゲットのよく似た部分を部分的な領域を見つけ出して、その情報をもとに姿勢を計算する
    - 部分的な類似性を計算するので、効率的な姿勢計算が可能になる
  
手順
1. 特徴点検出
    - ISS
    - 等間隔サンプリング
2. 特徴量記述
    - 特徴点に対してその特徴点らしさを表現する情報（アイデンティティ）を付与する処理
    - 特徴点周りの形状をもとに計算した多次元ベクトルを特徴量とすることが一般的
3. 対応点探索
    - 物体モデルと入力シーン間で物理的に同一の地点を指す座標同士の対応を得るようにしたい
    - 両特徴量間のノルムが最も小さい特徴点のペアを対応点にする
        - 誤った点に対応した時
        1. 特徴量間のノルムに閾値を設ける
            - だが点群にはセンサノイズやオクルージョンによる部分的な欠損が生じるため、閾値設定が難しい
        2. 双方向チェック
            - ソースからターゲットへのベストマッチ、ターゲットからソースへのベストマッチが一致すれば対応点としてみなす
        3. Ratio Test
            - 最近傍のノルムが他と比べて際立って小さいかどうかを調べる
            - 第一と第二で比を取って、閾値以下の場合に対応点とする
4. 姿勢計算
    - RANSAC(RANdom SAmple Consensus)

In [8]:
#特徴点検出
#今回は等間隔サンプリング：Voxel Grid Filterを使用

#特徴量記述
#今回はFPTH特徴量を利用する

import copy

path = "../3rdparty/Open3D/examples/test_data/ICP/"
source = o3d.io.read_point_cloud(path+"cloud_bin_0.pcd")
target = o3d.io.read_point_cloud(path+"cloud_bin_1.pcd")

source.paint_uniform_color([0.5, 0.5, 1])
target.paint_uniform_color([1.0, 0.5, 0.5])
initial_trans = np.identity(4)
initial_trans[0, 3] = -3.0

"""
点群を画面表示するための関数
source : 点群をリストとして渡す
target : 点群をリストとして渡す
transformation : 姿勢変換のための変換行列
"""
def draw_registration_result(source, target, transformation):
    pcds = list()
    for s in source:
        temp = copy.deepcopy(s)
        pcds.append(temp.transform(transformation))
    pcds += target
    o3d.visualization.draw_geometries(pcds, zoom=0.3199,
                                     front = [0.024, -0.225, -0.073],
                                     lookat = [0.488, 1.722, 1.556],
                                     up = [0.047, -0.972, 0.226])
    
draw_registration_result([source], [target], initial_trans)

In [9]:
"""
二つの点群から、特徴点を検出し、特徴量を計算する
pcd : 点群データ
voxel_size : Voxel Grid Filterのボクセルサイズ→検出する特徴点の間隔
"""
def keypoint_and_future_extraction(pcd, voxel_size):
    #間引いた点群を特徴点とする
    keypoints = pcd.voxel_down_sample(voxel_size)
    
    """法線を計算"""
    viewpoint = np.array([0., 0., 0.], dtype='float64')
    #間引いたデータの倍の半径を指定
    radius_normal = 2.0 * voxel_size
    #半径内で最大30点を使って計算する
    keypoints.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    #原点に法線ベクトルが向くように法線方向に反転処理を適用している
    keypoints.orient_normals_towards_camera_location(viewpoint)
    
    """特徴量を計算"""
    #特徴点の間隔の5倍の距離を限度とした球領域とする
    radius_feature = 5.0 * voxel_size
    #球領域に含まれる点の近いものから順に最大100点を選択する
    feature = o3d.pipelines.registration.compute_fpfh_feature(
        keypoints,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return keypoints, feature

voxel_size = 0.1
s_kp, s_feature = keypoint_and_future_extraction(source, voxel_size)
t_kp, t_feature = keypoint_and_future_extraction(target, voxel_size)

In [10]:
s_kp.paint_uniform_color([0, 1, 0])
t_kp.paint_uniform_color([0, 1, 0])
draw_registration_result([source, s_kp], [target, t_kp], initial_trans)

In [11]:
#対応点探索 : Ratio Test
#特徴量ベクトルを取り出して、(n, 33)行列を作成
np_s_feature = s_feature.data.T
np_t_feature = t_feature.data.T

#corrsは対応点のセットを保存するための変数。ソース、ターゲット双方の特徴点のインデックスを保持する
corrs = o3d.utility.Vector2iVector()
threshold = 0.9
#ソースの特定の特徴点の特徴量と、ターゲットの特徴量のL2ノルムを計算する。
#ノルムが小さいものから１位と2位の比を計算して、threshold以下であれば、正しい対応点セットとみなして、ソース、ターゲットのインデックスを保存する
for i, feat in enumerate(np_s_feature):
    distance = np.linalg.norm(np_t_feature - feat, axis=1)
    nearest_idx = np.argmin(distance)
    dist_order = np.argsort(distance)
    ratio = distance[dist_order[0]] / distance[dist_order[1]]
    if ratio < threshold:
        corr = np.array([[i], [nearest_idx]], np.int32)
        corrs.append(corr)
    
print("対応点セットの数 : ", (len(corrs)))

対応点セットの数 :  270


#### 姿勢計算
- 対応点探索によって得られた対応点を使って、ソースをターゲットに位置合わせする変換を推定する
    - ただし、対応点セットには誤りを含む場合があることに注意する
    - ここで頑健な推定法として、有名なRANSAC(RANdom SAmple Consensus)を利用する
  
#### RANSAC
- 点群処理以外にも広く使われている外れ値に対して、頑健な推定法の一種
- 外れ値が含まれた観測値から、その影響を抑えつつ、モデルパラメータを推定する
  
方法
1. サンプリング
    - ランダムに数個の計測点を選択し、モデルパラメータを推定する
    - サンプリングの処理の時に、外れ値を除いたデータのみを引き当てることを期待している
2. 評価
    - 得られたモデルパラメータの良さを評価する

In [16]:
"""
Open3Dのvisualizerで表示可能な直線群を作成する
"""
def create_lineset_from_correspondences(corrs_set, pcd1, pcd2, 
                                       transformation=np.identity(4)):
    pcd1_temp = copy.deepcopy(pcd1)
    pcd1_temp.transform(transformation)
    corrs = np.asarray(corrs_set)
    np_points1 = np.array(pcd1_temp.points)
    np_points2 = np.array(pcd2.points)
    points = list()
    lines = list()
    
    for i in range(corrs.shape[0]):
        points.append(np_points1[corrs[i, 0]])
        points.append(np_points2[corrs[i, 1]])
        lines.append([2*i, (2*i)+1])
        
    colors = [np.random.rand(3) for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines)
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

line_set = create_lineset_from_correspondences(corrs, s_kp, t_kp,
                                              initial_trans)
draw_registration_result([source, s_kp],
                        [target, t_kp, line_set],
                        initial_trans)

In [17]:
"""
外れ値を含む対応点を使って、姿勢を計算したため、ずれが目立つ
"""
#全ての点を使って、姿勢計算をする
#二つの対応の取れた点群の二乗誤差を最小化する変換行列を算出する。スケーリングを含めた変換の推定が可能。ここでFalseはスケーリングが1ということ
trans_ptp = o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
#変換行列を算出
trans_all = trans_ptp.compute_transformation(s_kp, t_kp, corrs)
draw_registration_result([source], [target], trans_all)

#### RANSACの実装
1. サンプリング ： 全ての対応点からあらかじめ決めておいた個数の対応点を選択し、変換行列を計算する。
- TransformationEstimationPointToPoint()を利用する
2. 評価 ： 得られた変換行列の妥当性を評価する。ソース側の点群を変換行列によって姿勢変換し、ターゲット側の点群との距離を計算する。
- この値があらかじめ決めておいた、マージンより小さい場合は、その対応点をインライアとして判定する。またインライアの対応点一点あたりの距離の平均値を計算する。この値が小さいほど、良い変換行列ということ。
  
1,2を繰り返して、最も良い変換行列を最終結果とする
  
引数の説明
- s_kp, t_kp, corrs : RANSACのために必要な材料
    - それぞれソース側の特徴点、ターゲット側の特徴点、対応点探索によって得られた対応点のインデックスのリスト
- distance_threshold　: インライアと判定するマージンの閾値
- ransac_n : 姿勢変換行列の計算のためにサンプリングする対応点の個数
- checkers : 枝切り処理に使われる条件
    - サンプリングと評価の間に簡単な条件を設定することでによって、すでに調べる必要のない（外れ値を含んだ）サンプルを除外すること。そしてこれによって高速化を図る
    - EdgeLength : サンプリングした対応点の配置の関係性を評価する
        - 片方の点群内での対応転換の距離のこと
        - ソースないで2点選び、同様にターゲット内で２点選び、それぞれの２点の対応点の距離を比較した時にどれくらい類似するかで枝切りする
            - 同一の点であれば、距離が似たような値になるはず
    - Distance : 変換行列によって、サンプリングした対応点(ransac_n点)を変換し、距離が近いかどうかを判定する
        - 近ければ、有望な変換行列としてみなす
 - criteria : RANSACの終了条件を指定する
     - 第一引数 : 試行回数の最大数
     - 第ニ引数 : 早期終了の時に使う引数
  
resultの要素とその内容
- correspondence_set : インライアと判定された対応点のインデックスのリスト
- fitness : インライア数/対応点数の値、大きいほどいい
- inlier_rmse : インライアの平均二乗誤差、小さいほどよい
- transformation : 4×4の変換行列

In [19]:
distance_threshold = voxel_size * 1.5
result = o3d.pipelines.registration.registration_ransac_based_on_correspondence(
    s_kp, t_kp, corrs,
    distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n = 3,
    checkers = [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ],
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
    )

In [21]:
#対応点の可視化
line_set = create_lineset_from_correspondences(result.correspondence_set, 
                                              s_kp, t_kp, initial_trans)
draw_registration_result([source, s_kp],
                        [target, t_kp, line_set],
                        initial_trans)

In [22]:
#実際にマッチングして可視化
draw_registration_result([source], [target], result.transformation)

In [24]:
#Open3Dには、対応点探索とRANSACによる姿勢計算をまとめて実行する方法がある
distance_threshold = voxel_size * 1.5
result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    s_kp, t_kp, s_feature, t_feature, True, 
    distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    ransac_n = 3,
    checkers = [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    ],
    criteria = o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999)
    )

In [25]:
#実際にマッチングして可視化
draw_registration_result([source], [target], result.transformation)

## 5.3 一般物体の姿勢推定

#### 割愛する

## 5.4 プリミティブ検出

- プリミティブ検出
    - 平面や球などの単純な図形を検出する処理
    - 　机の上に並べた対象物を検出したい場合、事前に平面を検出してから、その部分の点群を削除してしまえば、ここの物体を簡単に単離（検出）できる
    - シーン理解のためのかなり強力な前処理として利用できる

In [31]:
#平面の検出
"""
segment_plane()ではRANSACによる平面検出を実行している
RANSACによって、平面の方程式ax + by + cz + d = 0で表す。そのため、モデルパラメータはa, b, c, dとなる。
考え方は、今までと同様で最も点群に近い平面を見つけるということ。そのため、あるランダムに取った3点の平面とその他の点群との距離を計算して、インライアの閾値をクリアすればカウントする。　
評価方法は、インライア点数/総点数でインライアの平均誤差が最小になるものを採用する。

引数
distance_theshold : RANSACの評価処理で利用される。平面のインライアとして判定するための距離の閾値。単位はメートル
ransac_n : RANSACのサンプリング処理で利用される。この点数から平面のパラメータを計算する。
num_iterations : RANSACのサンプリング処理と評価処理の繰り返し回数
"""

pcd = o3d.io.read_point_cloud("../data/tabletop_scene.ply")
o3d.visualization.draw_geometries([pcd])
#plane_model : 平面パラメータ
#inliers : 元の点群における、平面上の点のインデックスのリスト
plane_model, inliers = pcd.segment_plane(distance_threshold=0.005,
                                        ransac_n=3,
                                        num_iterations=500)

[a, b, c, d] = plane_model
print(f"Plane equation : {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0") 

#点群を平面のものとそれ以外で色分けする
plane_cloud = pcd.select_by_index(inliers)
plane_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([plane_cloud, outlier_cloud])

Plane equation : 0.02x + 0.74y + 0.67z + -0.31 = 0


In [42]:
#球の検出(Open3Dには実装されていない)
import numpy as np
"""
球の方程式は(x-a)^2 + (y-b)^2 + (z-c)^2 = r^2となるから、a, b, c, dの4つがパラメータになる。
球の方程式に4つの３次元点群を入力してあげて、連立方程式を解く。
評価方法は、平面の検出と同様に、インライア点数/総点数でインライアの平均誤差が最小になるものを採用する。
"""
def ComputerSphereCoefficient(p0, p1, p2, p3):
    A = np.array([p0-p3, p1-p3, p2-p3])
    p3_2 = np.dot(p3, p3)
    b = np.array([(np.dot(p0, p0) - p3_2) / 2,
                  (np.dot(p1, p1) - p3_2) / 2,
                  (np.dot(p2, p2) - p3_2) / 2])
    coeff = np.zeros(3)
    try:
        #連立方程式を解く
        ans = np.linalg.solve(A, b)
    except:
        print("Error!! Matrix rank is", np.linalg.matrix_rank(A))
        print("Return", coeff)
        pass
    else:
        tmp = p0 - ans
        r = np.sqrt(np.dot(tmp, tmp))
        #球の方程式のパラメータa, b, c, rが格納される
        coeff = np.append(ans, r)
        
    return coeff

In [43]:
"""
球の中心とある点群との距離を計算して、半径から閾値に収まるものをインライアとしてカウントする。
pcd : 入力点群
coeff : 球のパラメータ
distance_th : 距離の閾値
fitness : 出力、モデルである球の当てはめのよさ　
inlier_dist : インライアの平均誤差
inliers : インライア点群のインデックスリスト
"""
def EvaluateSphereCoefficient(pcd, coeff, distance_th=0.01):
    fitness = 0
    inlier_dist = 0
    inliers = None
    
    #点群中の全ての点に対する球面との距離を一気に計算（ブロードキャスト）
    dist = np.abs(np.linalg.norm(pcd - coeff[:3], axis=1) - coeff[3])
    n_inlier = np.sum(dist < distance_th)
    if n_inlier != 0:
        fitness = n_inlier / pcd.shape[0]
        inlire_dost = np.sum((dist < distance_th) * dist) / n_inlier
        inliers = np.where(dist < distance_th)[0]
        
    return fitness, inlier_dist, inliers

In [44]:
#繰り返し演算の実装（より良いパラメータを探す）
pcd = outlier_cloud
np_pcd = np.asarray(pcd.points)
ransac_n = 4 #点群から選択する点数は球だと4
num_iterations = 1000 #RANSACの試行回数
distance_th = 0.005 #モデルと点群の距離の閾値
max_radius = 0.05 #検出する球の半径の最大値

#初期化
best_fitness = 0 #モデルの当てはめの良さ、インライア点数/全点数
best_inlier_dist = 10000.0 #インライア点の平均距離
best_inliers = None #元の点群におけるインライアのインデックス
best_coeff = np.zeros(4) #モデルパラメータ

for n in range(num_iterations):
    c_id = np.random.choice(np_pcd.shape[0], 4, replace=False)
    coeff = ComputerSphereCoefficient(
                                    np_pcd[c_id[0]], np_pcd[c_id[1]], np_pcd[c_id[2]], np_pcd[c_id[3]])
    if max_radius < coeff[3]:
        continue
    fitness, inlier_dist, inliers = EvaluateSphereCoefficient(np_pcd, coeff, distance_th)
    if (best_fitness < fitness) or ((best_fitness == fitness) and (inlier_dist < best_inlier_dist)):
        best_fitness = fitness
        best_inlier_dist = inlier_dist
        best_inliers = inliers
        best_coeff = coeff
        print(f"Update: Fitness = {best_fitness:.4f}, Inlier_dist = {best_inlier_dist:.4f}")
        
if best_coeff.any() != False:
    print(f"Sphere equation:  (x - {best_coeff[0]:.2f})^2 + (y - {best_coeff[1]:.2f})^2 + (z - {best_coeff[2]:.2f})^2) = {best_coeff[3]:.2f}^2")
else:
    print(f"No sphere detected.")

Update: Fitness = 0.3306, Inlier_dist = 0.0000
Update: Fitness = 0.3320, Inlier_dist = 0.0000
Sphere equation:  (x - 0.12)^2 + (y - 0.07)^2 + (z - 0.34)^2) = 0.04^2


In [45]:
#インライアを青に着色して可視化する
sphere_cloud = pcd.select_by_index(best_inliers)
sphere_cloud.paint_uniform_color([0, 0, 1.0])
outlier_cloud = pcd.select_by_index(best_inliers, invert=True)
o3d.visualization.draw_geometries([sphere_cloud, outlier_cloud])

In [46]:
#平面と球を同時に検出。そしてそれぞれを色で分ける
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=best_coeff[3])
mesh_sphere.compute_vertex_normals()
mesh_sphere.paint_uniform_color([0.3, 0.3, 0.7])
mesh_sphere.translate(best_coeff[:3])
o3d.visualization.draw_geometries([mesh_sphere] + [sphere_cloud+plane_cloud+outlier_cloud])

## 5.5 セグメンテーション

- オーバーセグメンテーション　
    - 認識したい物体より細かいパーツ単位でセグメンテーションすること
- DBSCAN(Density Based Spatial Clustering of Applications with Noise)
    - 点群データのオーバーセグメンテーション
    - ２点間のユークリッド距離に基づいたクラスタリングを行う手法
    - ２点間距離がある閾値以内である点同士を同じクラスタと認識し、それ以外を外れ値とする
    - 全て点に対して、コア点か境界点か外れ値かのどれかで分ける
        - これを繰り返す

In [50]:
import sys
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

#filename = sys.argv[1]
filename = "../3rdparty/Open3D/examples/test_data/fragment.pcd"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

labels = np.array(pcd.cluster_dbscan(eps=0.02, min_points=10))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / max(max_label, 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd], zoom=0.8,
                                 front=[-0.04999, -0.1659, -0.8499],
                                 lookat=[2.1813, 2.0619, 2.0999],
                                 up=[0.1204, -0.9852, 0.1215])

Loading a point cloud from ../3rdparty/Open3D/examples/test_data/fragment.pcd
PointCloud with 113662 points.
point cloud has 7 clusters
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
